In [ ]:
!pip install qiskit qiskit-ibm-runtime pylatexenc qiskit_aer

In [ ]:
# 불켜기 문제 1단계 version 1

In [ ]:
adj_rooms = [[1,3],   #  i번째 방과 j번째 방이 인접하면
            [0,2,4],  #  adj_rooms[i] == j
            [1,5],
            [0,4,6],
            [1,3,5,7],
            [2,4,8],
            [3,7],
            [4,6,8],
            [5,7]]
# 3x3 형태로 램프 on/off 상태를 1/0 으로 표시
def show_lights(rooms, str=''):
    print(str)
    for i in range(3):
      for j in range(3):
        print(rooms[i*3+j], end=' ')
      print()
    print()
    return
# 스위치의 동작을 단계별로 표시함
def switch_lights(switch,rooms):
    show_lights(rooms, 'initial status :')
    # 스위치를 동작하여 램프를 반전
    for i in range(9):
        if switch[i] == 1:
        # i번 스위치가 1이면 i번 방과 인접한 방의 상태를
        # 0<->1 로 반전
            rooms[i] = 1-rooms[i]
            for c in adj_rooms[i]:
                rooms[c] = 1-rooms[c]
            show_lights(rooms, f'after switch {i} is on :')
    return

In [ ]:
# problem specification
problem = [0, 0, 0, 1, 0, 1, 1, 1, 0]
# proposed solution
switch_answer = [0, 0, 1, 1, 0, 0, 1, 0, 1]

print('switches = ', switch_answer)
print('problem = ', problem)

#apply the answer to the problem
switch_lights(switch_answer,problem)

solved = True
for r in problem:
  if r != 1:
    solved = False
    print('The problem is not solved.')
    break
if solved:
  print('lights are all turned on.')
  print('press the switches ', end='')
  for i in range(len(switch_answer)):
    if switch_answer[i] == 1:
      print(i, end=',')
  print(' to turn all the lights.')

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
# 불켜기 문제 1단계 그로버 회로 초기화
qr = QuantumRegister(9,name='r') # room 의 불켜짐=1, 불꺼짐=0 을 표현하는 양자 레지스터
qx = QuantumRegister(9,name='x') # 스위치의 동작=1, 동작않음=0 을 표현하는 양자 레지스터
qy = QuantumRegister(1,name='y') # 그로버 회로의 Uf|x,y>=|x,f(x)XOR y> 의 y (오라클 큐비트)
solution = ClassicalRegister(9,'c')  # 스위치의 동작여부를 표현하는 qx 를 측정할 고전 레지스터

circuit = QuantumCircuit(qy, qr, qx, solution)
# y 를 |-> 로 초기화
circuit.x(qy)
circuit.h(qy)

# i번 방이 켜져있을 경우 (rooms[i]==1) qr[i]를 |1> 상태로 초기화
rooms = [0, 0, 0, 1, 0, 1, 1, 1, 0]
for i in range(9):
  if  rooms[i] == 1 :
    circuit.x(qr[i])

# x = |+++++++++>
circuit.h(qx)

circuit.draw(output='mpl')

In [ ]:
adj_rooms = [[1,3],   #  i번째 방과 j번째 방이 인접하면
            [0,2,4],  #  adj_rooms[i] == j
            [1,5],
            [0,4,6],
            [1,3,5,7],
            [2,4,8],
            [3,7],
            [4,6,8],
            [5,7]]

def switch_func(switch,rooms):
# switch[i] 큐비트이 1이면 i-번째 방과 인접한 방들의 불을 반전시키는 회로를 반환하는 함수
    circuit = QuantumCircuit(switch,rooms)
    for i in range(9):
        circuit.cx(switch[i], rooms[i])
        for c in adj_rooms[i]:
            circuit.cx(switch[i], rooms[c])
        circuit.barrier()
    return circuit

# switch_func 함수를 호출하여 circuit 회로에 추가
circuit.compose(switch_func(qx,qr),qx[:9]+qr[:9],inplace=True)

# y = f(x) XOR y in Uf
# qr[i] 가 모두 1이면 (=모든 방의 불이 켜졌으면) f(x)=1
circuit.mcx(qr, qy)

circuit.draw(output='mpl', fold=-1)


In [ ]:
# diffusion circuit for n qubits
def diffuser_circuit(n):
    x = QuantumRegister(n,'x')
    qc = QuantumCircuit(x) # n 큐비트 회로 객체 생성
    qc.h(x)                # 모든 큐비트에 H
    qc.x(x)                # 모든 큐비트에 X

    qc.barrier()
    qc.h(x[0])             # x0 (target 큐비트) 에만 H
    qc.mcx(x[1:n], x[0])   # 제어 큐비트 x[1,2,...,n-1], 타겟 큐비트 x[0] 인 다중제어 X gate
    qc.h(x[0])             # x0 (target 큐비트) 에만 H
    qc.barrier()           # HXH=Z 이기 때문에 이 결과는 다중제어-Z gate 와 같다.

    qc.x(x)                # 모든 큐비트에 X
    qc.h(x)                # 모든 큐비트에 H

    qc.name = "D"          # 회로 이름 설정
    return qc


In [ ]:
def lights_oracle(qy,qr,qx):
  # 그로버 회로의 오라클 연산 Uf|x,y>=|x,y XOR f(x)> 를 구현하는 회로를 생성
  circuit = QuantumCircuit(qy, qr, qx)
  circuit.compose(switch_func(qx,qr),qx[:9]+qr[:9],inplace=True)
  circuit.mcx(qr, qy)
  circuit.compose(switch_func(qx,qr),qx[:9]+qr[:9],inplace=True)
  circuit.name = 'Oracle'
  return circuit

# 불켜기 문제 1단계 그로버 회로 초기화
qr = QuantumRegister(9,name='r') # room 의 불켜짐=1, 불꺼짐=0 을 표현하는 양자 레지스터
qx = QuantumRegister(9,name='x') # 스위치의 동작=1, 동작않음=0 을 표현하는 양자 레지스터
qy = QuantumRegister(1,name='y') # 그로버 회로의 Uf|x,y>=|x,f(x)XOR y> 의 y (오라클 큐비트)
solution = ClassicalRegister(9,'c')  # 스위치의 동작여부를 표현하는 qx 를 측정할 고전 레지스터
circuit = QuantumCircuit(qy, qr, qx, solution)

# y 를 |-> 로 초기화
circuit.x(qy)
circuit.h(qy)

# i번 방이 켜져있을 경우 (rooms[i]==1) qr[i]를 |1> 상태로 초기화
rooms = [0, 0, 0, 1, 0, 1, 1, 1, 0]
for i in range(9):
  if  rooms[i] == 1 :
    circuit.x(qr[i])

# x = |+++++++++>
circuit.h(qx)

circuit.compose(lights_oracle(qy,qr,qx),qy[:]+qr[:]+qx[:],inplace=True)
circuit.barrier()
circuit.compose(diffuser_circuit(9), qx, inplace=True)
circuit.draw(output='mpl', fold=-1,plot_barriers=False)

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
# 불켜기 문제 그로버 회로 초기화
qr = QuantumRegister(9,'r') # room 의 불켜짐=1, 불꺼짐=0 을 표현하는 양자 레지스터
qx = QuantumRegister(9,'x') # 스위치의 동작=1, 동작않음=0 을 표현하는 양자 레지스터
qy = QuantumRegister(1,'y') # 그로버 회로의 Uf|x,y>=|x,f(x)XOR y> 의 y (오라클 큐비트)
c = ClassicalRegister(9,'c')  # 스위치의 동작여부를 표현하는 qx 를 측정할 고전 레지스터

circuit = QuantumCircuit(qy, qr, qx, c)
# y 를 |-> 로 초기화
circuit.x(qy)
circuit.h(qy)

# i번 방이 켜져있을 경우 (rooms[i]==1) qr[i]를 |1> 상태로 초기화
rooms = [0,0,0, 1,0,1, 1,1,0]
for i in range(9):
  if  rooms[i] == 1 :
    circuit.x(qr[i])

# x = |+++++++++>
circuit.h(qx)

for _ in range(17): #  그로버 반복 17회
  circuit.append(lights_oracle(qy,qr,qx),range(19))
  circuit.append(diffuser_circuit(9), qx)

circuit.measure(qx,reversed(c))
circuit.draw(output='mpl', fold=-1)

In [ ]:
# AerSimulator 를 이용하여 circuit 회로를 측정한 결과를 출력
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_aer import AerSimulator
aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
isa_circuit = pm.run(circuit)
sampler = Sampler(mode=aer_sim)
job = sampler.run([isa_circuit], shots=1)
result = job.result()
count = result[0].data.c.get_counts()

from qiskit.visualization import plot_distribution
plot_distribution(count)

In [ ]:
answer = list(count.keys())[0]

print(answer)
switch_answer = [int(answer[i]) for i in range(9)]
problem = rooms.copy()
print('switches = ', switch_answer)
print('problem = ', problem)

switch_lights(switch_answer,problem)

solved = True
for r in problem:
  if r != 1:
    solved = False
    break
if solved:
  print('lights are all turned on.')
  print('press the switches ', end='')
  for i in range(len(switch_answer)):
    if switch_answer[i] == 1:
      print(i, end=',')
  print(' to turn all the lights.')
else :
  print('Some lights are turned off. Try again,')

In [ ]:
# 불켜기 문제 1단계 version 2

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
# 불켜기 문제 1단계 그로버 회로 초기화 - version 2
qr = QuantumRegister(9,'r') # room 의 불켜짐=1, 불꺼짐=0 을 표현하는 양자 레지스터
qx = QuantumRegister(9,'x') # 스위치의 동작=1, 동작않음=0 을 표현하는 양자 레지스터
qy = QuantumRegister(1,'y') # 그로버 회로의 Uf|x,y>=|x,f(x)XOR y> 의 y (오라클 )
c = ClassicalRegister(3,'c') # qx[0,1,2] 를 측정할 3-bit 고전 레지스터

circuit = QuantumCircuit(qy, qr, qx, c)
# y 를 |-> 로 초기화
circuit.x(qy)
circuit.h(qy)

# i번 방이 켜져있을 경우 (rooms[i]==1) qr[i]를 |1> 상태로 초기화
rooms = [0, 0, 0, 1, 0, 1, 1, 1, 0]
for i in range(9):
  if  rooms[i] == 1 :
    circuit.x(qr[i])

# x[0,1,2] = |+++>
circuit.h(qx[:3])

circuit.barrier()
circuit.draw(output='mpl', fold=-1)

In [ ]:
adj_rooms = [[1,3],   #  i번째 방과 j번째 방이 인접하면
            [0,2,4],  #  adj_rooms[i] == j
            [1,5],
            [0,4,6],
            [1,3,5,7],
            [2,4,8],
            [3,7],
            [4,6,8],
            [5,7]]
# switch[i] 이 1이면 i-번째 방과 인접한 방들의 불을 반전시키는 함수
def switch_func3(switch,rooms):
    circuit = QuantumCircuit(switch,rooms)
    for row in range(3):
    # row 번째 줄의 스위치를 동작
        for i in range(3*row, 3*row+3):
            circuit.cx(switch[i], rooms[i])
            for c in adj_rooms[i]:
                circuit.cx(switch[i], rooms[c])
        circuit.barrier()
        if row == 2: break
        # row 번째 줄의 점등 상태의 반대로 row+1 번째 줄 스위치를 설정 (마지막 줄 제외)
        for j in range(3*row, 3*row+3):
            circuit.cx(rooms[j], switch[j+3],ctrl_state=0)
        circuit.barrier()
    circuit.name = 'Switch3'
    return circuit

# switch_func3 함수를 호출하여 circuit 회로에 추가
circuit.compose(switch_func3(qx,qr),qx[:9]+qr[:9],inplace=True)
circuit.draw(output='mpl', fold=-1,plot_barriers=False)

In [ ]:
def uncompute_switch_func3(switch,rooms):
# switch_func3 의 uncomputation 을 위하여
# switch_func3 의 for loop 들을 반대 순서로 실행
    circuit = QuantumCircuit(switch,rooms)
    for row in range(2,-1,-1):
        for i in range(3*row,3*row+3):
            circuit.cx(switch[i], rooms[i])
            for c in adj_rooms[i]:
                circuit.cx(switch[i], rooms[c])
        circuit.barrier()
        if row == 0: break
        for j in range(3*(row-1),3*row):
            circuit.cx(rooms[j], switch[j+3],ctrl_state=0)
        circuit.barrier()
    circuit.name = '-Switch3'
    return circuit

# y = f(x) XOR y in Uf
# qr[i] 가 모두 1이면 (=모든 방의 불이 켜졌으면) f(x)=1
circuit.mcx(qr[6:9], qy)
circuit.barrier();
# uncomputation : Uf|x,y>=|x,y XOR f(x)>
circuit.compose(uncompute_switch_func3(qx,qr),qx[:9]+qr[:9],inplace=True)

circuit.draw(output='mpl', fold=-1, plot_barriers=True)


In [ ]:
def lights_oracle3(qy,qr,qx):
# 그로버 회로의 Uf|x,y>=|x,y XOR f(x)> 를 구현하는 회로를 생성 (version 2)
  circuit = QuantumCircuit(qy, qr, qx)
  circuit.append(switch_func3(qx,qr),qx[:]+qr[:])
  circuit.mcx(qr[6:9], qy)
  circuit.append(uncompute_switch_func3(qx,qr),qx[:]+qr[:])
  circuit.name = 'Oracle3'
  return circuit

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
# 불켜기 문제 1단계 그로버 회로 초기화 - version 2
qr = QuantumRegister(9,'r') # room 의 불켜짐=1, 불꺼짐=0 을 표현하는 양자 레지스터
qx = QuantumRegister(9,'x') # 스위치의 동작=1, 동작않음=0 을 표현하는 양자 레지스터
qy = QuantumRegister(1,'y') # 그로버 회로의 Uf|x,y>=|x,f(x)XOR y> 의 y (오라클 )
c = ClassicalRegister(3,'c') # qx[:3] 를 측정할 3-bit 고전 레지스터

circuit = QuantumCircuit(qy, qr, qx, c)
# y 를 |-> 로 초기화
circuit.x(qy)
circuit.h(qy)

# i번 방이 켜져있을 경우 (rooms[i]==1) qr[i]를 |1> 상태로 초기화
rooms = [0, 0, 0, 1, 0, 1, 1, 1, 0]
for i in range(9):
  if  rooms[i] == 1 :
    circuit.x(qr[i])

# x[0,1,2] = |+++>
circuit.h(qx[:3])

for _ in range(2):  # n=3 일 때 그로버 반복 2회
  circuit.append(lights_oracle3(qy,qr,qx),range(19))
  circuit.append(diffuser_circuit(3), qx[:3])

circuit.measure(qx[:3],reversed(c))
circuit.draw(output='mpl', fold=-1)

In [ ]:
# AerSimulator 를 이용하여 circuit 회로를 측정한 결과를 출력
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_aer import AerSimulator
aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
isa_circuit = pm.run(circuit)
sampler = Sampler(mode=aer_sim)
job = sampler.run([isa_circuit], shots=1)
result = job.result()
count = result[0].data.c.get_counts()

from qiskit.visualization import plot_distribution
plot_distribution(count)

In [ ]:
def switch3to8(switch,rooms):
  for row in range(3):
    # row 번째 줄의 스위치를 동작
    for i in range(3*row, 3*row+3):
        if switch[i] == 1:
            rooms[i] = 1-rooms[i]
            for c in adj_rooms[i]:
                rooms[c] = 1-rooms[c]
    show_lights(rooms, f'after switches in the {row}-th row are applied :')

    if row == 2: return

    # row 번째 줄의 점등 상태의 반대로 row+1 번째 줄 스위치를 설정 (마지막 줄 제외)
    for i in range(3*row, 3*row+3):
        switch[i+3] = 1-rooms[i]
    print('switch : ', switch)

In [ ]:

measured = list(count.keys())[0] 
print('measured value = ', measured)
answer = measured + '0000000'

switch_answer = [int(answer[i]) for i in range(9)]
problem = rooms.copy()
print('switches = ', switch_answer)
print('problem = ', problem)
show_lights(problem, 'initial status :')
# apply the answer to the problem
switch3to8(switch_answer,problem)

solved = True
for r in problem:
  if r != 1:
    solved = False
    break
if solved:
  print('lights are all turned on.')
  print('press the switches ', end='')
  for i in range(len(switch_answer)):
    if switch_answer[i] == 1:
      print(i, end=',')
  print(' to turn all the lights.')
else :
  print('Some lights are turned off. Try again,')

In [ ]:
# 불켜기 문제 2단계 

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

# 2-비트 qp 이진 입력에 대해 9-비트 qr 이진 출력의 진리표를
# 양자 회로로 구현하여 반환하는 함수
def problem_map(qp, qr, problems):
  circuit = QuantumCircuit(qp,qr)
  for p in range(4):
    circuit.barrier()
    rooms = problems[p]
    if p//2 == 0:    # p1 == 0
      circuit.x(qp[1])
    if p%2 == 0:     # p0 == 0
      circuit.x(qp[0])
    for i in range(9):
      if rooms[i] == '1':
        circuit.ccx(qp[0],qp[1],qr[i])
    if p//2 == 0:    # p1 == 0
      circuit.x(qp[1])
    if p%2 == 0:     # p0 == 0
      circuit.x(qp[0])
  circuit.name = "Problem"
  return circuit

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
# 불켜기 문제 그로버 회로 2단계 초기화
#problems=  ['111000100','101000110','101111001','100000100']
problems = ['000101110','111011110','000110111','101001101']

qp = QuantumRegister(2,name='p') # 문제 번호 00,01,10,11 을 표현하는 양자 레지스터
qr = QuantumRegister(9,name='r') # room 의 불켜짐=1, 불꺼짐=0 을 표현하는 양자 레지스터
circuit = QuantumCircuit(qp,qr)

# p[0,1] = |++>
circuit.h(qp)

circuit.compose(problem_map(qp,qr,problems),qp[0:2]+qr[0:9],inplace=True)

circuit.measure_all()
circuit.draw(output='mpl', fold=-1)

In [ ]:
# AerSimulator 를 이용하여 circuit 회로를 측정한 결과를 출력
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_aer import AerSimulator
aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
isa_circuit = pm.run(circuit)
sampler = Sampler(mode=aer_sim)
job = sampler.run([isa_circuit], shots=10000)
result = job.result()
count = result[0].data.meas.get_counts()

from qiskit.visualization import plot_distribution
plot_distribution(count)

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

# n 비트 이진 스트링 q_str 의 1의 갯수를 세어 4비트 2진수로 표현하는 회로
def count1(n,q_str,q_cnt):
  circuit = QuantumCircuit(q_str, q_cnt)
  for i in range(n):
    circuit.mcx([q_str[i],q_cnt[0],q_cnt[1],q_cnt[2]],q_cnt[3])
    circuit.mcx([q_str[i],q_cnt[0],q_cnt[1]],q_cnt[2])
    circuit.ccx(q_str[i],q_cnt[0],q_cnt[1])
    circuit.cx(q_str[i],q_cnt[0])
  circuit.name = 'Count'
  return circuit

qc = QuantumRegister(4,name='count') 
qx = QuantumRegister(9,name='x') 
circuit = QuantumCircuit(qx,qc)

circuit.x(qx[:3])
circuit.barrier()
circuit.compose(count1(9,qx,qc),qx[:]+qc[:],inplace=True)
circuit.measure_all()
circuit.draw(output='mpl', fold=-1)

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
# 불켜기 문제를 하위 문제로 하는 그로버 알고리즘 2단계 회로
problems = ['000101110','111011110','000110111','101001101']

qp = QuantumRegister(2, name='p') # 문제 번호 00,01,10,11 을 표현하는 양자 레지스터
qr = QuantumRegister(9,name='r') # room 의 불켜짐=1, 불꺼짐=0 을 표현하는 양자 레지스터
qx = QuantumRegister(9,name='x') # 스위치의 동작=1, 동작않음=0 을 표현하는 양자 레지스터
qc = QuantumRegister(4,name='c') # 스위치 중 1의 갯수를 이진수로 표현하는 양자 레지스터
qy = QuantumRegister(1,name='y') # 그로버 회로의 Uf|x,y>=|x,f(x)XOR y> 의 y (오라클 큐비트)
solution = ClassicalRegister(2,name='solution') # qp를 측정할 2-bit 고전 레지스터

# c1 : 상위 문제의 f() 함수를 계산하기 위해 qp 에 대한 하위 문제를 그로버 반복 2회로 구현하고
#      얻어진 해 (qx) 의 1의 갯수를 qc 에 2진수로 표현하는 회로
c1 = QuantumCircuit(qy, qp, qr, qx, qc)

c1.append(problem_map(qp,qr,problems),qp[:]+qr[:])

for _ in range(2):  # n=3 불켜기 문제 그로버 반복 2회
  c1.append(lights_oracle3(qy,qr,qx), qy[:]+qr[:]+qx[:])
  c1.append(diffuser_circuit(3), qx[:3])

c1.append(switch_func3(qx,qr), qx[:]+qr[:])
c1.append(count1(9,qx,qc),qx[:]+qc[:])
c1.name = 'LightsOn+'

inv_c1 = c1.inverse()                   # inv_c1 : c1 회로의 역계산 회로
inv_c1.name = 'LightsOn-'

c1.draw(output='mpl')

In [ ]:
inv_c1.draw(output='mpl')

In [ ]:
# 불켜기 문제를 하위 문제로 하는 그로버 알고리즘 2단계 회로
problems = ['000101110','111011110','000110111','101001101']

qp = QuantumRegister(2, name='p') # 문제 번호 00,01,10,11 을 표현하는 양자 레지스터
qr = QuantumRegister(9,name='r') # room 의 불켜짐=1, 불꺼짐=0 을 표현하는 양자 레지스터
qx = QuantumRegister(9,name='x') # 스위치의 동작=1, 동작않음=0 을 표현하는 양자 레지스터
qc = QuantumRegister(4,name='c') # 스위치 중 1의 갯수를 이진수로 표현하는 양자 레지스터
qy = QuantumRegister(1,name='y') # 그로버 회로의 Uf|x,y>=|x,f(x)XOR y> 의 y (오라클 큐비트)
solution = ClassicalRegister(2,name='solution') # qp를 측정할 2-bit 고전 레지스터
circuit = QuantumCircuit(qy, qp, qr, qx, qc, solution)
# y 를 |-> 로 초기화
circuit.x(qy)
circuit.h(qy)

# p[0,1] = |++>
circuit.h(qp)

# x[0,1,2] = |+++>
circuit.h(qx[:3])

# 상위 문제 그로버 오라클 회로
circuit.append(c1,range(25))      # c1 회로를 이용하여 p 의 해를 구하고 1의 갯수를 세는 회로
circuit.mcx(qc[:],qy,ctrl_state='0101') # y XOR f(p) 를 계산
circuit.append(inv_c1,range(25))  # c1의 역계산 회로를 사용하여 Uf|p,y>->|p,f(p)XOR y>

# 상위 문제 그로버 확산자 회로
circuit.append(diffuser_circuit(2), qp)

circuit.measure(qp,solution)
circuit.draw(output='mpl', fold=-1)

In [ ]:
# AerSimulator 를 이용하여 circuit 회로를 측정한 결과를 출력
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_aer import AerSimulator
aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
isa_circuit = pm.run(circuit)
sampler = Sampler(mode=aer_sim)
job = sampler.run([isa_circuit], shots=1)
result = job.result()
count = result[0].data.solution.get_counts()

from qiskit.visualization import plot_distribution
plot_distribution(count)